In [1]:
import pandas as pd
import json
import string

In [2]:
chart_name = "ZAv1"
chart_name = "globalv1"

In [3]:
# all_files_za = pd.read_csv("Classification/all_files_" + chart_name + ".csv")

In [4]:
all_files_za = pd.read_csv("Classification/all_files_" + chart_name + ".csv")

In [5]:
all_files_za.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'lower_main_artist', 'Position over Time'],
      dtype='object')

In [6]:
all_files_za = all_files_za[["End Date", "artist_names", "main_artist", "track_name", "streams", 'source', "Year"]]

In [7]:
def groupbysum(all_files, year, relevantcols, grouplist, sumcol, sortcol, isascending, isYear, filepath):
    if(isYear):
        za2022 = all_files[all_files["Year"] == year]
    else:
        za2022 = all_files

#     print(za2022.shape)

    za2022df = za2022[relevantcols].groupby(grouplist)[sumcol].apply(sum)

    za2022dsummed = za2022df.reset_index()

    zasummedsorted = za2022dsummed.sort_values(by=sortcol, ascending=isascending)

    zasummedsortedfin = zasummedsorted.reset_index().drop(["index"], axis=1)

    zasummedsortedfin.to_csv(filepath,index=False)
    
    return zasummedsortedfin

In [8]:
relevantcols = ['main_artist','streams', "source"]
grouplist = ['source']
sumcol = 'streams'
sortcol = 'streams'
isascending = False
isYear = False
filepath = "Classification/"+chart_name+"sourcefiles.csv"
zasummedsortedstreamssource = groupbysum(all_files_za, 2022, relevantcols, grouplist, sumcol, sortcol, isascending, isYear, filepath)

In [9]:
# zasummedsortedfin

In [10]:
def groupbyvalue(za2022, idcol, relevantcols, groupcol, countdistinctcol):
    za2022df = za2022[relevantcols].groupby([groupcol]).agg({countdistinctcol: pd.Series.nunique})
    new2022df = pd.DataFrame(za2022df)
    new2022df.columns = ["count_" + countdistinctcol]
    
    new2022df = new2022df.reset_index()
    
    new2022dfunique = new2022df[[idcol, "count_" + countdistinctcol]]
    
    new2022dfunique = new2022dfunique.drop_duplicates()
    
    return new2022dfunique

def sortvaluesbyorder(new2022dfunique, countdistinctcolname, isascending):
    zacountsorted = new2022dfunique.sort_values(by=countdistinctcolname, ascending=isascending)
    zacountsorted = zacountsorted.reset_index()
    zacountsorted.drop(["index"], inplace=True, axis=1)
    zacountsorted = zacountsorted.dropna()
    
    return zacountsorted

In [11]:
relevantcols = ['main_artist', 'track_name', 'streams', "source"]
groupcol = "source"
countdistinctcol = "main_artist"

idcol = "source"
new2022dfunique = groupbyvalue(all_files_za, idcol, relevantcols, groupcol, countdistinctcol)

print(list(new2022dfunique.columns), new2022dfunique.shape)

countdistinctcolname = "count_" + countdistinctcol

zacountsortedartistspersource = sortvaluesbyorder(new2022dfunique, countdistinctcolname, False)

# zacountsorted.to_csv("Classification/sortedsourcesartists_ZA2022.csv", index=False)

['source', 'count_main_artist'] (838, 2)


In [12]:
relevantcols = ['main_artist', 'track_name', 'streams', "source"]
groupcol = "source"
countdistinctcol = "track_name"

idcol = "source"
new2022dfunique = groupbyvalue(all_files_za, idcol, relevantcols, groupcol, countdistinctcol)

print(list(new2022dfunique.columns), new2022dfunique.shape)

countdistinctcolname = "count_" + countdistinctcol

zacountsortedtrackspersource = sortvaluesbyorder(new2022dfunique, countdistinctcolname, False)


['source', 'count_track_name'] (838, 2)


In [13]:
# zasummedsortedfin.shape, zacountsortedartistspersource.shape, zacountsortedtrackspersource.shape

In [14]:
zasourcedf1 = zasummedsortedstreamssource.merge(zacountsortedartistspersource, left_on=["source"], right_on=["source"], how="inner")

In [15]:
zasourcedf = zasourcedf1.merge(zacountsortedtrackspersource, left_on=["source"], right_on=["source"], how="inner")

In [16]:
zasourcedf

,source,streams,count_main_artist,count_track_name
0,Columbia,37711187917,56,230
1,Republic Records,27555882345,22,145
2,Rimas Entertainment LLC,18294105384,9,90
3,Atlantic Records UK,17993141099,13,73
4,Atlantic Records,13445634616,38,79
...,...,...,...,...
833,Ninetone Records,3125372,1,1
834,Major Tom's / Atlantic,3111623,1,1
835,MCI,3052833,1,1
836,VÉRITÉ,2753947,1,1


In [17]:
zasourcedf.to_csv("Classification/" + chart_name + "countsortedartistspersource.csv", index=False)

##### Find South African artists

In [18]:
# Read the south african file
files_za_artists = pd.read_csv("Classification/JustSouthAfricansOnZAChart.csv")

In [19]:
files_za_artists.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'Lower Artist'],
      dtype='object')

In [20]:
zasources = files_za_artists[["source"]].drop_duplicates()

In [21]:
justzasourcedf = zasourcedf.merge(zasources, left_on=["source"], right_on=["source"], how="right")

In [22]:
# justzasourcedf.to_csv("Classification/" + chart_name + "_sourcedetails.csv", index=False)